# Fall 2022 Data Science Intern Challenge - Shopify

## Question 1

Given some sample data, write a program to answer the following:
> On Shopify, we have exactly 100 sneaker shops, and each of these shops sells only one model of shoe. We want to do some analysis of the average order value (AOV). When we look at orders data over a 30 day window, we naively calculate an AOV of $3145.13. Given that we know these shops are selling sneakers, a relatively affordable item, something seems wrong with our analysis. 
> - Think about what could be going wrong with our calculation. Think about a better way to evaluate this data. 
> - What metric would you report for this dataset?
> - What is its value?


### Upload and clean the data

First, let's import useful packages:

In [1]:
# Linear algebra
import pandas as pd
import numpy as np

# Data visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Statistics library
from statsmodels.graphics.gofplots import qqplot

Then, load and preview the dataset:

In [2]:
data = pd.read_csv("shopify-internship-challenge-dataset.csv")
data.head(3)

,order_id,shop_id,user_id,order_amount,total_items,payment_method,created_at
0,1,53,746,224,2,cash,2017-03-13 12:36:56
1,2,92,925,90,1,cash,2017-03-03 17:38:52
2,3,44,861,144,1,cash,2017-03-14 4:23:56


We can see this dataset provides information about each order, especially the **order_amount** which will be useful to compute the Average Order Value (AOV) we are looking for. 

Before looking at dataset statistics, let's first make sure it doesn't have null or duplicate values:

In [3]:
# Null values percentage for each column
data.isnull().sum().sort_values(ascending=False)/len(data) 

created_at        0.0
payment_method    0.0
total_items       0.0
order_amount      0.0
user_id           0.0
shop_id           0.0
order_id          0.0
dtype: float64

In [4]:
data.duplicated().sum() # Number of duplicated rows

0

The dataset has no null or duplicated rows, we can start exploring it further.

### Think about what could be going wrong with our calculation. Think about a better way to evaluate this data

Let's take a closer look at the order_amount column:

In [5]:
data.order_amount.describe()

count      5000.000000
mean       3145.128000
std       41282.539349
min          90.000000
25%         163.000000
50%         284.000000
75%         390.000000
max      704000.000000
Name: order_amount, dtype: float64

- The average order amount is equal to \$3 145.13 which is exactly the "naively" calculated AOV stated in the instructions.  
Indeed, it seems pretty expensive when we know orders items are just pairs of shoes (a relatively affordable item).


- The maximum order_amount is \$704 000 and the standard deviation is \\$41 282.  
These are very large numbers, indicating the presence of outliers driving the mean up.

  
- The third quartile shows that 75% of orders amounts fall under \$390 which sounds more realistic. It is way lower than the mean and the maximum value, indicating that outliers are extremely high values.

Before taking any decision on removing outliers from the AOV, let's look at the integrity of orders with the maximum value:

In [18]:
maximum_amount_orders = data[data.order_amount == 704000]
maximum_amount_orders.reset_index()

,index,order_id,shop_id,user_id,order_amount,total_items,payment_method,created_at
0,15,16,42,607,704000,2000,credit_card,2017-03-07 4:00:00
1,60,61,42,607,704000,2000,credit_card,2017-03-04 4:00:00
2,520,521,42,607,704000,2000,credit_card,2017-03-02 4:00:00
3,1104,1105,42,607,704000,2000,credit_card,2017-03-24 4:00:00
4,1362,1363,42,607,704000,2000,credit_card,2017-03-15 4:00:00
5,1436,1437,42,607,704000,2000,credit_card,2017-03-11 4:00:00
6,1562,1563,42,607,704000,2000,credit_card,2017-03-19 4:00:00
7,1602,1603,42,607,704000,2000,credit_card,2017-03-17 4:00:00
8,2153,2154,42,607,704000,2000,credit_card,2017-03-12 4:00:00
9,2297,2298,42,607,704000,2000,credit_card,2017-03-07 4:00:00


There are already 17 of them with the maximum amount of \$704 000. 
All of them look very similar,  which is suprising on a 30-day window.  
They were all created at 4 AM, done by the same user_id 607 on the same shop_id 42.  
These elements are good indicators of fraudulent orders that we would need to report to the Fraud or Risk team.  
Now with that in mind, let's look into another metric than the mean to get useful insights out of this dataset

### What metric would you report for this dataset?

- We need a metric that is robust to outliers.  
I would report the median because it takes the middle number in the sorted values, thus not impacted by outliers.

### What is its value?

In [6]:
median_order_value = data.order_amount.median()
print(f'The median order value is {int(median_order_value)}.')

The median order value is 284.


## Question 2

Given some sample data, write a program to answer the following:
> For this question you’ll need to use SQL. Please use queries to answer the following questions. Paste your queries along with your final numerical answers below.
> - How many orders were shipped by Speedy Express in total?
> - What is the last name of the employee with the most orders?
> - What product was ordered the most by customers in Germany?


### How many orders were shipped by Speedy Express in total?

After exploring the database relationships, we know:  
- the Orders table displays the ShipperID for each order
- the Shippers table gives us the ShipperID for each shipper name

We can therefore join both tables on ShipperID and count the number of order shipped by Speedy Express:

``` mysql
SELECT
    count(distinct Orders.OrderID) as orders_count
    
FROM Orders

LEFT JOIN Shippers
    ON Shippers.ShipperID =  Orders.ShipperID
    
WHERE Shippers.ShipperName = 'Speedy Express'


--> **Speedy Express has shipped 54 orders**

### What is the last name of the employee with the most orders?

After exploring the database relationships, we know:  
- the Employees table displays the last name and EmployeeID for each employee
- the Orders table gives us the EmployeeID for each shipper name

We can therefore join both tables on EmployeeID, count the number of order for each employee last name, sort values in descending order and limit to the first result.

``` mysql
SELECT
	Employees.LastName,
    count(distinct Orders.OrderID) AS orders_count
    
FROM Employees

LEFT JOIN Orders
	ON Orders.EmployeeID = Employees.EmployeeID

GROUP BY 1
ORDER BY 2 desc
LIMIT 1 

````


### What product was ordered the most by customers in Germany?

After exploring the database relationships, we know:  
- the OrderDetails table gives the productID and quantity of each product that was ordered for each order.
- the Orders table gives the CustomerID for each order.
- the Customers table gives the Country and Customer ID for each customer.
- the Products table gives the name and ProductID of each product

We can therefore join Customers and Orders on CustomerID, join Orders and OrderDetails on OrderID, OrderDetails and Products on ProductID, filter on Customers in Germany only, sum the quantity bought for each product, sort values in a descending order and limit to the first result.

``` mysql
SELECT
	OrderDetails.ProductID,
    Products.ProductName,
    sum(OrderDetails.Quantity) as quantity
    
FROM OrderDetails

LEFT JOIN Orders
	ON Orders.OrderID = OrderDetails.OrderID
   
LEFT JOIN Customers
	on Customers.CustomerID = Orders.CustomerID
    
LEFT JOIN Products
	on Products.ProductID = OrderDetails.ProductID

WHERE Customers.country = 'Germany'

GROUP BY 1, 2
ORDER BY 3 DESC
LIMIT 1

````


--> **The most ordered product by customers in Germany is Boston Crab Meat with 160 items**